<a href="https://colab.research.google.com/github/cwattsnogueira/bikeease-ad-generator/blob/main/genai_inc_unit6_capstone02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Essentials and Applications of Generative AI: Incremental Capstone
Carllos Watts-Nogueira

Due: 13/Sep/2025

Overview

BikeEase has successfully implemented various AI-powered solutions for demand forecasting, customer review analysis, and image classification. As they continue to grow, they aim to automate certain tasks using Large Language Models (LLMs), particularly in marketing and advertising generation to attract more customers and increase engagement.

To achieve this, BikeEase plans to develop a Generative AI-powered system that can automatically create engaging and persuasive advertisements based on bike specifications, discount offers, and promotional themes. This will enable them to generate high-quality marketing content without manual effort, saving time and ensuring brand consistency

Project Statement

Develop a Generative AI-powered advertisement generation system using LLMs and LangChain to create compelling promotional content for BikeEase’s rental services

## BikeEase Ad Generator — Capstone

###  Task 1: Understanding Generative AI & LLMs
> In this section, here: explore how Large Language Models (LLMs) can be used for automated marketing and how LangChain helps integrate them into applications.

In [14]:
# Install required libraries
!pip install langchain langchain-community transformers gradio

In [33]:
# Import core libraries
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline
import gradio as gr

###  Task 2: Designing the Ad Generation Pipeline

> here: design a pipeline that accepts user inputs (bike specs, discount, theme) and uses a Hugging Face model to generate structured, persuasive ads aligned with BikeEase’s branding.


In [34]:
# Load the local Hugging Face model
def load_model(model_name):
    generator = pipeline("text-generation", model=model_name, max_length=300, do_sample=True)
    return HuggingFacePipeline(pipeline=generator)

In [35]:
# Build LangChain pipeline with structured prompt
def build_chain(model_name):
    prompt = PromptTemplate(
        input_variables=["specs", "discount", "theme"],
        template="""
[Header]
Introducing the ultimate ride: {specs}

[Subheader]
Experience innovation and comfort in every pedal.

[Body]
Enjoy exclusive savings: {discount}
Theme: {theme}

[CTA]
Book now and ride free for the first hour.

[Hashtag]
#BikeEase #RideSmart #EcoAdventure

[Footer]
Visit BikeEase.com or scan the QR code to start your journey.
"""
    )
    llm = load_model(model_name)
    return LLMChain(prompt=prompt, llm=llm)

###  Task 3: Building the LLM-Based Ad Generator

> Here: use LangChain to manage prompt engineering and integrate a local Hugging Face model. This allows us to generate ads without relying on external APIs.


In [42]:
# Available models for generation
available_models = [
    "google/flan-t5-small",
    # "Adnane10/AdsGeniusAI",
    # "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # "mistralai/Mistral-7B-Instruct-v0.1"
]

In [37]:
# Generate ad and handle errors
def generate_ad(specs, discount, theme, model_name):
    if not specs or not discount or not theme:
        return "Please fill in all fields.", "Missing input data."

    try:
        chain = build_chain(model_name)
        ad = chain.run({"specs": specs, "discount": discount, "theme": theme})
        score, feedback = evaluate_ad(ad, specs, discount, theme)
        return ad, f"Score: {score}/6\nFeedback: {'; '.join(feedback) if feedback else 'Excellent!'}"
    except Exception as e:
        return "Error generating ad.", f"Details: {str(e)}"

###  Task 4: Evaluation and Optimization

> Here: evaluate the generated ads for quality, persuasiveness, and relevance. This includes checking for key sections and matching user inputs.



In [38]:
# Evaluate the quality of the generated ad
def evaluate_ad(ad_text, specs, discount, theme):
    score = 0
    feedback = []

    if "[Header]" in ad_text and specs in ad_text:
        score += 1
    else:
        feedback.append("Missing [Header] or specs.")

    if "[Subheader]" in ad_text:
        score += 1
    else:
        feedback.append("Missing [Subheader].")

    if "[Body]" in ad_text and discount in ad_text and theme.lower() in ad_text.lower():
        score += 1
    else:
        feedback.append("Body section incomplete or missing inputs.")

    if "[CTA]" in ad_text:
        score += 1
    else:
        feedback.append("Missing [CTA].")

    if "[Hashtag]" in ad_text:
        score += 1
    else:
        feedback.append("Missing [Hashtag].")

    if "[Footer]" in ad_text:
        score += 1
    else:
        feedback.append("Missing [Footer].")

    return score, feedback

###  Final Interface: Gradio App for Public Presentation

> Here: build a user-friendly interface using Gradio so users can input campaign details and receive instant ad generation and evaluation.


In [43]:
# Launch Gradio app
with gr.Blocks() as demo:
    gr.Markdown("##  BikeEase Ad Generator")
    specs = gr.Textbox(label="Bike Specifications")
    discount = gr.Textbox(label="Discount Offer")
    theme = gr.Textbox(label="Marketing Theme")
    model_name = gr.Dropdown(choices=available_models, label="Choose Model", value=available_models[0])
    output = gr.Textbox(label="Generated Advertisement")
    feedback = gr.Textbox(label="Evaluation")

    generate_btn = gr.Button("Generate Ad")
    generate_btn.click(fn=generate_ad, inputs=[specs, discount, theme, model_name], outputs=[output, feedback])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d416fe8e4be0d1cabc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
